In [23]:
import pyodbc
import pandas as pd
from time import gmtime, strftime
from alpha_vantage.timeseries import TimeSeries
import time
import QuantLib as ql
from utilities.date_utilities import *


In [30]:
connection = pyodbc.connect('Driver=SQL Server;'
                      'Server=DESKTOP-0CQQR1E\SQLEXPRESS;'
                      'Database=securities_master;'
                      'Trusted_Connection=yes;')
cursor=connection.cursor()
ts = TimeSeries(key='NHXLC1G323WY6E6I', output_format='pandas')

In [31]:

def check_update(ticker):
    """
    check if a  ticker's history is None or the latest update is performed more (excluding) than 1 data before

    :param ticker:
    :return: -999 for not None, float for business days since last update
    """
    global connection
    proceed_update=False
    
    

    
    data = pd.read_sql_query(
        f"select top 1 *  from securities_hist where ticker='{ticker}' order by eff_date desc", connection)

    if len(data)<1: # first-time update
        return -999
    else:
        current_time = pd.to_datetime(strftime("%Y-%m-%d %H:%M:%S", gmtime()))
        last_updated_time = data["eff_date"][0]
        business_day_since_update = business_day_between(last_updated_time,current_time)
        return business_day_since_update,data["eff_date"][0]
    





In [32]:
check_update("A")

(0, Timestamp('2021-06-16 00:00:00'))

In [33]:


def insert_sec_full(ticker,sec_id,sleep_time=14):
    global connection,cursor,ts
    b_days_since_last_eff, last_eff = check_update(ticker) 

    if b_days_since_last_eff==-999: # security is new, loading all data
        print(f"{ticker} does not exist in the database, fetching full history...")
        data = ts.get_daily_adjusted(ticker, outputsize='full')[0]
        time.sleep(sleep_time)
  
    elif b_days_since_last_eff>=2: # security exists but not up to date
        print(f"{ticker} is {b_days_since_last_eff} days behind, fetching new data...")
        data = ts.get_daily_adjusted(ticker, outputsize='full')[0]
        data = data[:last_eff].iloc[:-1]
        time.sleep(sleep_time)
    else:
        print(f"{ticker} is up to date, no action needed.")
        return 
    
    
    if len(data)>0:
        
        
        for idx in range(len(data)):
            record=data.iloc[idx]
            curr_time=strftime("%Y-%m-%d %H:%M:%S", gmtime())
            insert_str = f"""
            INSERT INTO securities_hist
            (
            sec_id,
            ticker,
            instrument,
            eff_date,
            open_price,
            high_price,
            low_price,
            close_price,
            adj_close_price,
            volume,
            dividend,
            split,
            created_date,
            last_updated_date)
            values (

            '{sec_id}',
            '{ticker}',
            '{'Common Stock'}',
            '{record.name}',
            {record['1. open']},
            {record['2. high']},
            {record['3. low']},
            {record['4. close']},
            {record['5. adjusted close']},
            {record['6. volume']},
            {record['7. dividend amount']},
            {record['8. split coefficient']},
            '{curr_time}',
            '{curr_time}'

            )
            """

            cursor.execute(insert_str)
            connection.commit()
        print(f"{ticker} updated.")







In [34]:


static_sec_info = pd.read_sql_query("select * from securities", connection)


# update tickers

for  ticker,sec_id in zip(sorted(static_sec_info.ticker.values),sorted(static_sec_info.sec_id.values)):
    try:

        insert_sec_full(ticker,sec_id)
    except Exception:
        print(f"fatal error {ticker}")
        time.sleep(15)
        continue

        


connection.close()



A is up to date, no action needed.
AAL is 93 days behind, fetching new data...
AAL updated.
AAP is 93 days behind, fetching new data...
AAP updated.
AAPL is 93 days behind, fetching new data...
AAPL updated.
ABBV is 93 days behind, fetching new data...
ABBV updated.
ABC is 93 days behind, fetching new data...
ABC updated.
ABMD is 93 days behind, fetching new data...
ABMD updated.
ABT is 93 days behind, fetching new data...
ABT updated.
ACN is 93 days behind, fetching new data...
ACN updated.
ADBE is 93 days behind, fetching new data...
ADBE updated.
ADI is 93 days behind, fetching new data...
ADI updated.
ADM is 93 days behind, fetching new data...
ADM updated.
ADP is 93 days behind, fetching new data...
ADP updated.
ADSK is 93 days behind, fetching new data...
ADSK updated.
AEE is 93 days behind, fetching new data...
AEE updated.
AEP is 93 days behind, fetching new data...
AEP updated.
AES is 93 days behind, fetching new data...
AES updated.
AFL is 93 days behind, fetching new data...

DVA updated.
DVN is 93 days behind, fetching new data...
DVN updated.
DXC is 93 days behind, fetching new data...
DXC updated.
DXCM is 93 days behind, fetching new data...
DXCM updated.
EA is 93 days behind, fetching new data...
EA updated.
EBAY is 93 days behind, fetching new data...
EBAY updated.
ECL is 93 days behind, fetching new data...
ECL updated.
ED is 93 days behind, fetching new data...
ED updated.
EFX is 93 days behind, fetching new data...
EFX updated.
EIX is 93 days behind, fetching new data...
EIX updated.
EL is 93 days behind, fetching new data...
EL updated.
EMN is 93 days behind, fetching new data...
EMN updated.
EMR is 93 days behind, fetching new data...
EMR updated.
ENPH is 93 days behind, fetching new data...
ENPH updated.
EOG is 93 days behind, fetching new data...
EOG updated.
EQIX is 93 days behind, fetching new data...
EQIX updated.
EQR is 93 days behind, fetching new data...
EQR updated.
ES is 93 days behind, fetching new data...
ES updated.
ESS is 93 days beh

LW updated.
LYB is 94 days behind, fetching new data...
LYB updated.
LYV is 94 days behind, fetching new data...
LYV updated.
MA is 94 days behind, fetching new data...
MA updated.
MAA is 94 days behind, fetching new data...
MAA updated.
MAR is 94 days behind, fetching new data...
MAR updated.
MAS is 94 days behind, fetching new data...
MAS updated.
MCD is 94 days behind, fetching new data...
MCD updated.
MCHP is 94 days behind, fetching new data...
MCHP updated.
MCK is 94 days behind, fetching new data...
MCK updated.
MCO is 94 days behind, fetching new data...
MCO updated.
MDLZ is 94 days behind, fetching new data...
MDLZ updated.
MDT is 94 days behind, fetching new data...
MDT updated.
MET is 94 days behind, fetching new data...
MET updated.
MGM is 94 days behind, fetching new data...
MGM updated.
MHK is 94 days behind, fetching new data...
MHK updated.
MKC is 94 days behind, fetching new data...
MKC updated.
MKTX is 94 days behind, fetching new data...
MKTX updated.
MLM is 94 days 

TFC updated.
TFX is 94 days behind, fetching new data...
TFX updated.
TGT is 94 days behind, fetching new data...
TGT updated.
TJX is 94 days behind, fetching new data...
TJX updated.
TMO is 94 days behind, fetching new data...
TMO updated.
TMUS is 94 days behind, fetching new data...
TMUS updated.
TPR is 94 days behind, fetching new data...
TPR updated.
TRMB is 94 days behind, fetching new data...
TRMB updated.
TROW is 94 days behind, fetching new data...
TROW updated.
TRV is 94 days behind, fetching new data...
TRV updated.
TSCO is 94 days behind, fetching new data...
TSCO updated.
TSLA is 94 days behind, fetching new data...
TSLA updated.
TSN is 94 days behind, fetching new data...
TSN updated.
TT is 94 days behind, fetching new data...
TT updated.
TTWO is 94 days behind, fetching new data...
TTWO updated.
TWTR is 94 days behind, fetching new data...
TWTR updated.
TXN is 94 days behind, fetching new data...
TXN updated.
TXT is 94 days behind, fetching new data...
TXT updated.
TYL is

In [7]:
connection.close()

ProgrammingError: Attempt to use a closed connection.